# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В таблице 12 столбцов. Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

**Вывод**

В данных видны следующие проблемы:
1. В столбце `days_employed` есть артефакты: отрицательные и очень больше положительные числа. Возможно, положительные числа соответствуют трудовому стажу в часах, а отрицательные достаточно взять по модулю;
2. Уровни образования написаны разными регистрами;
3. В столбцах `days_employed` и `total_income` есть пропущенные значения. Т.к. в этих столбцах одинаковое количество пропусков, можно предположить, что трудовой стаж и ежемесячный доход пропущены у одних и тех же клиентов;
4. Трудовой стаж и ежемесячный доход — вещественные числа;
5. Минимальное значение в столбце `children` — -1, а максимальное — 20. Возможно, это опечатки при вводе чисел "1" и "2";
6. Минимальное значение в столбце `dob_years` — 0. Скорее всего, нулём заполнили пропуски.

Для проверки гипотез нам понадобятся столбцы `children`, `family_status`, `family_status_id`, `debt`, `total_income` и `purpose`, поэтому нужно исправить ошибки в этих столбцах, если они есть.

## Шаг 2. Предобработка данных

### Обработка пропусков

Подсчитаем количество пропущенных значений и проверим, в одних и тех же строках пропущены значения трудового стажа и ежемесячного дохода или нет.

In [5]:
print(df.isna().sum())
print('Количество строк с пропусками в обоих столбцах:', df[df['days_employed'].isna() & df['total_income'].isna()].shape[0])

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Количество строк с пропусками в обоих столбцах: 2174


Предположение подтвердилось. Трудовой стаж и ежемесячный доход пропущены у одних и тех же клиентов.

Устраним артефакты в столбце `days_employed`. Сначала создадим сводную таблицу по каждому типу занятости, содержащую следующую информацию:
1. Количество строк, соответствующих данному типу занятости;
2. Средний трудовой стаж;
3. Количество положительных значений в столбце `days_employed`.

In [6]:
df_pivot = df.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})
df_pivot = df_pivot.rename(columns={'count': 'number_of_rows', 'mean':'mean_days_employed', '<lambda_0>':'number_of_rows_>0'})
df_pivot

days_employed                                     
                number_of_rows mean_days_employed number_of_rows_>0
income_type                                                        
безработный                  2      366413.652744               2.0
в декрете                    1       -3296.759962               0.0
госслужащий               1312       -3399.896902               0.0
компаньон                 4577       -2111.524398               0.0
пенсионер                 3443      365003.491245            3443.0
предприниматель              1        -520.848083               0.0
сотрудник                10014       -2326.499216               0.0
студент                      1        -578.751554               0.0

Заметим, что трудовой стаж безработных и пенсионеров — большие положительные числа. Трудовой стаж для остальных категорий указан в виде отрицательного числа. Заменим отрицательные числа положительными.

In [7]:
df['days_employed'] = abs(df['days_employed'])

Узнаем средний возраст пенсионеров и безработных.

In [8]:
print('Средний возраст пенсионеров:', df[df['income_type'] == 'пенсионер']['dob_years'].mean())
print('Средний возраст безработных:', df[df['income_type'] == 'безработный']['dob_years'].mean())

Средний возраст пенсионеров: 59.06301867219917
Средний возраст безработных: 38.0


Узнаем, у людей с каким типом занятости есть пропущенные значения в столбцах.

In [9]:
df[df['days_employed'].isna()]['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Проверим предположение о том, что трудовой стаж пенсионеров записан в часах. Для этого переведём его в дни, а потом выведем средний трудовой стаж в годах.

In [10]:
df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df['days_employed'] / 24
df[df['income_type'] == 'пенсионер']['days_employed'].mean() / 365

41.6670652106006

Средний возраст пенсионеров — 59 лет, а средний трудовой стаж — 41 год. Значит, трудовой стаж пенсионеров действительно мог быть записан в днях.

Средний трудовой стаж у безработных больше, чем у пенсионеров, а возраст меньше. Значит, трудовой стаж безработных заполнен по какому-то другому принципу, но это не имеет для нас значения, т.к. у безработных нет пропусков в данных, а столбец с трудовым стажем не важен для проверки гипотез.

Перейдём к устранению пропусков в столбце `days_employed` и `total_income`. На место пропущенных значений подставим медианное значение трудового стажа для каждого соответсвующего типа занятости.

In [11]:
for type in df['income_type'].unique():
    columns_to_fill = ['days_employed', 'total_income']
    for column in columns_to_fill:
        median_value = df.loc[df['income_type'] == type, column].median()
        df.loc[df['income_type'] == type, column] = df[column].fillna(median_value)

Исправим странные значения в столбце `children`. Сначала узнаем, какие значения есть в этом столбце, затем устраним артефакты.

In [12]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [13]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2

Возраст некоторых клиентов равен 0. Заменим его на медианное значение по соответствующему типу занятости.

In [14]:
for i in df['income_type'].unique():
    median_dob = int(df.loc[df['income_type'] == i, 'dob_years'].median())
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == i), 'dob_years'] = median_dob

**Вывод**

В столбцах `days_employed` и `total_income` были обнаружены пропущенные значения. Возможно, в этих столбцах есть пропуски, потому что некоторые клиенты банка решили скрыть свой трудовой стаж и доход. Пропуски были заполнены медианным значением по каждой группе, т.к. в данных могут встречаться значения, сильно отличающиеся от других, которые сдвинули бы среднее. 

### Замена типа данных

In [15]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

**Вывод**

Методом astype() удобно переводить столбцы с данными в нужный тип. С помощью этого метода перевели значения в столбцах с трудовым стажем и доходом в целочисленный тип.

### Обработка дубликатов

Выведем уникальные значения в столбцах `education` и ` family_status`.

In [16]:
print(df['education'].unique())
print(df['family_status'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


Данные об образовании и семейном статусе написаны разными регистрами. Приведём их к нижнему регистру.

In [17]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

Перейдём к удалению дубликатов. Сначала подсчитаем их количество.

In [18]:
df.duplicated().sum()

71

Теперь удалим дубликаты с обновлением индексов и удалением старых.

In [19]:
df = df.drop_duplicates().reset_index(drop = True)

**Вывод**

Скорее всего, дубликаты появились из-за повторного внесения данных некоторых клиентов и ошибок при заполнении колонок. Дубликаты могут повлиять на выводы при проверке гипотез, поэтому их важно удалять.

### Лемматизация

Выведем уникальные цели получения кредита

In [20]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим столбец `purpose_category`, в котором будут храниться основные цели получения кредита: "недвижимость", "свадьба", "автомобиль" и "образование".

In [21]:
from pymystem3 import Mystem
m = Mystem()
def purpose_category(purpose):
    purpose_lemma = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemma or 'жилье' in purpose_lemma:
        return('недвижимость')
    if 'образование' in purpose_lemma:
        return('образование')
    if 'автомобиль' in purpose_lemma:
        return('автомобиль')
    if 'свадьба' in purpose_lemma:
        return('свадьба')
    return('другое')
    
df['purpose_category'] = df['purpose'].apply(purpose_category)

**Вывод**

Благодаря лемматизации мы смогли создать столбец с короткой формулировкой цели кредита, который поможет нам в дальнейшем при проверке гипотез.

### Категоризация данных

Классифицируем клиентов по количеству детей:
* 0 детей — бездетный;
* 1-2 ребенка;
* 3 и более детей — многодетный.

In [22]:
def children_category(number_of_children):
    if number_of_children == 0:
        return('бездетный')
    if number_of_children < 3:
        return('1-2 ребенка')
    return('многодетный')

df['clients_children_group'] = df['children'].apply(children_category)

Классифицируем клиентов по уровню дохода. Сначала разделим данные о доходах на квантили.

In [23]:
def quant(income):
    q = [0, 1/4, 1/2, 3/4, 1]
    return(income.quantile(q=q))

quant(df['total_income'])

0.00      20667.00
0.25     107623.00
0.50     142594.00
0.75     195820.25
1.00    2265604.00
Name: total_income, dtype: float64

Создадим классификацию, опираясь на полученные данные:
* до 107623 — низкий доход;
* до 142594 — средний доход;
* до 195820 — высокий доход
* больше 195820 — очень высокий доход.

In [24]:
def income_category(income):
    if income <= 107623:
        return('низкий доход')
    if income <= 142594:
        return('средний доход')
    if income <= 195820:
        return('высокий доход')
    return('очень высокий доход.')

df['total_income_group'] = df['total_income'].apply(income_category)

**Вывод**

Разделив данные на категории, мы сможем ответить на поставленные перед нами вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
children_pivot = df.pivot_table(index='clients_children_group', columns='debt', values='children', aggfunc='count')
children_pivot['%_with_debt'] = children_pivot[1] / (children_pivot[1] + children_pivot[0]) * 100
children_pivot

debt,0,1,%_with_debt
clients_children_group,,,
1-2 ребенка,6336,647,9.265359
бездетный,13028,1063,7.543822
многодетный,349,31,8.157895


**Вывод**

Бездетные клиенты реже допускают просрочки по кредитам, чем клиенты с детьми. Многодетных заемщиков слишком мало для того, чтобы можно было с уверенностью утверждать, что они чаще возвращают кредиты в срок, чем заемщики с 1-2 детьми. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
family_status_pivot = df.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
family_status_pivot['%_with_debt'] = family_status_pivot[1] / (family_status_pivot[1] + family_status_pivot[0]) * 100
family_status_pivot

debt,0,1,%_with_debt
family_status,,,
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
не женат / не замужем,2536,274,9.750890


**Вывод**

Клиенты, находящиеся или бывшие в браке чаще возвращают кредиты в срок, чем клиенты, не состоявшие в узаконенных отношениях.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
income_group_pivot = df.pivot_table(index='total_income_group', columns='debt', values='total_income', aggfunc='count')
income_group_pivot['%_with_debt'] = income_group_pivot[1] / (income_group_pivot[1] + income_group_pivot[0]) * 100
income_group_pivot

debt,0,1,%_with_debt
total_income_group,,,
высокий доход,4799,448,8.538212
низкий доход,4937,427,7.960477
очень высокий доход.,4981,383,7.140194
средний доход,4996,483,8.815477


**Вывод**

Клиенты со средним и высоким уровнем дохода допускают просрочки по кредиту чаще остальных.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
purpose_group_pivot = df.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
purpose_group_pivot['%_with_debt'] = purpose_group_pivot[1] / (purpose_group_pivot[1] + purpose_group_pivot[0]) * 100
purpose_group_pivot

debt,0,1,%_with_debt
purpose_category,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**

Клиенты, берущие кредит на недвижимость, оказались более ответственными и чаще возвращают кредит в срок. А заемщики, желающие получить кредит на образование или автомобиль, чаще остальных допускают просрочки по кредиту.

## Шаг 4. Общий вывод

Мы проверили четыре гипотезы и на основании данных сделали следующие выводы:

1. Наличие детей влияет на возврат кредита в срок. Бездетные клиенты допускают просрочки по кредитам реже всех. Среди бездетных клиентов только 7.5% имеют задолженности. У клиентов с детьми этот процент выше: 9.3% заемщиков с 1-2 детьми и 8.2% многодетных заемщиков. 
2. Семейное положение также влияет на наличие задолженностей. Среди клиентов банка, находящихся или бывших в официальных отношениях, наименьший процент должников: 6.6% вдовцов/вдов, 7.1% заемщиков в разводе, 7.5% женатых/замужних. Незамужние/неженатые клиенты и клиенты, состоящие в гражданском браке, имеют по 9.8% и 9.3% соответственно.
3. Также мы заметили зависимость между уровнем дохода и возвратом кредита в срок. Клиенты со средним доходом оказались самыми ненадежными: среди них 8.6% должников. Клиенты с низким доходом допускают просрочки в 7.9% случаев, клиенты с высоким доходом — в 7.1%. Клиенты с очень высоким доходом (больше 500 000) выплачивают кредиты исправнее всех: только 6.3% заемщиков имеют долги.
4. Цели получения кредита тоже влияют на выплату кредита в срок. Самыми ответственными заемщиками оказались люди, берущие кредит на недвижимость: 7.2% должников. За ними идут клиенты, желающие сыграть свадьбу: 8% должников. У желающих получить образование и купить автомобиль чаще возникают проблемы с погашением кредитов : 9.2% и 9,4% должников соответственно.

Отвечая на главный вопрос, можно сказать, что семейное положение и количество детей влияет на факт погашения кредита в срок. Самые ответственные заемщики — заемщики, находящиеся или находившиеся ранее в официальном браке и не имеющие детей. Наименее ответственными являются незамужние/неженатые клиенты с 1 или 2 детьми. 